## Data Collection

In [68]:
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer

def load_data(data_url):
    return pd.read_csv(data_url, sep='\t')

# URLs for train
train_data_url = 'https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/train.tsv'
train_data = load_data(train_data_url)
train_data.head(3)

,My favourite food is anything I didn't have to cook myself.,27,eebbqej
0,"Now if he does off himself, everyone will thin...",27,ed00q6i
1,WHY THE FUCK IS BAYLESS ISOING,2,eezlygj
2,To make her feel threatened,14,ed7ypvh


# Preprocessing

### Keep relevant columns and remove some rows

In [69]:
# Preprocess the data: rename columns and remove multiple emotions
def preprocess_data(data):
    # Define the header names
    header = ["comment", "emotion", "id"]
    # Rename the columns
    data.columns = header
    # Keep only the 'comment' and 'emotion' columns
    data = data[['comment', 'emotion']]
    # Filter out rows with multiple emotions (emotions separated by commas)
    data = data[data['emotion'].apply(lambda x: len(x.split(',')) == 1)]
    # Clean the 'emotion' column by keeping only digit characters and converting to integers
    data['emotion'] = data['emotion'].apply(lambda x: ''.join(filter(str.isdigit, str(x)))).astype(int)
    return data

preprocessed_data = preprocess_data(train_data)
preprocessed_data.head(3)

,comment,emotion
0,"Now if he does off himself, everyone will thin...",27
1,WHY THE FUCK IS BAYLESS ISOING,2
2,To make her feel threatened,14


### Filter top emotions for testing

In [70]:
# Filter the data to include only the top N most common emotions
def filter_top_emotions(data, top_n=2):
    # Count the occurrences of each emotion
    emotion_counts = data['emotion'].value_counts()
    # Identify the top N most common emotions
    top_emotions = emotion_counts.head(top_n).index
    # Create a mapping from the original emotion labels to new labels (0, 1, ...)
    label_mapping = {label: new_label for new_label, label in enumerate(top_emotions)}
    # Filter the data to include only the rows with the top N emotions
    filtered_data = data[data['emotion'].isin(top_emotions)].copy()
    # Apply the new label mapping to the 'emotion' column
    filtered_data.loc[:, 'emotion'] = filtered_data['emotion'].map(label_mapping)
    return filtered_data, label_mapping

filtered_train_data, label_mapping = filter_top_emotions(preprocessed_data)
filtered_train_data.head(3)

# Map the new labels to their original emotion names
emotions_dict = {
    0: "admiration", 1: "amusement", 2: "anger", 3: "annoyance", 4: "approval",
    5: "caring", 6: "confusion", 7: "curiosity", 8: "desire", 9: "disappointment",
    10: "disapproval", 11: "disgust", 12: "embarrassment", 13: "excitement", 14: "fear",
    15: "gratitude", 16: "grief", 17: "joy", 18: "love", 19: "nervousness",
    20: "optimism", 21: "pride", 22: "realization", 23: "relief", 24: "remorse",
    25: "sadness", 26: "surprise", 27: "neutral"
}
top_emotions_dict = {label_mapping[k]: emotions_dict[k] for k in label_mapping}

print("Top 2 most common emotions:", [top_emotions_dict[e] for e in sorted(top_emotions_dict)])

Top 2 most common emotions: ['neutral', 'admiration']


### Sample a fraction of the data for each emotion category

In [71]:
# Sample a fraction of the data for each emotion category
def sample_data(data, fraction=0.1):
    # Group the data by 'emotion' and sample a specified fraction of each group
    return data.groupby('emotion', group_keys=False).apply(lambda x: x.sample(frac=fraction)).reset_index(drop=True)

sampled_train_data = sample_data(filtered_train_data)
print("sampled_train_data.shape", sampled_train_data.shape)
sampled_train_data.head(3)

sampled_train_data.shape (1553, 2)


/var/folders/xd/gw_pmm5d23s8dm3h09tn28x00000gn/T/ipykernel_79046/1999367012.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return data.groupby('emotion', group_keys=False).apply(lambda x: x.sample(frac=fraction)).reset_index(drop=True)


,comment,emotion
0,Perhaps the person who named it didn't underst...,0
1,Then be happy for that. If you gave friends th...,0
2,Tell your wife you accept her offer to find an...,0


### Remove emotions

In [72]:
def remove_emotions(data, emotions_to_remove):
    # Remove specified emotions and create a copy to avoid SettingWithCopyWarning
    data = data[~data['emotion'].isin(emotions_to_remove)].copy()
    # Get sorted unique emotions
    unique_emotions = sorted(data['emotion'].unique())
    # Create a mapping from old label to new label
    label_mapping = {old_label: new_label for new_label, old_label in enumerate(unique_emotions)}
    # Map new labels to the data
    data.loc[:, 'emotion'] = data['emotion'].map(label_mapping)
    return data, label_mapping

remove_emotions_train_data, label_mapping = remove_emotions(sampled_train_data, emotions_to_remove=[1, 15])  # Remove 'amusement' (1) and 'gratitude' (15)
print("remove_emotions_train_data.shape", remove_emotions_train_data.shape)

remove_emotions_train_data.shape (1282, 2)


### Mapping from the 28 emotions to six broader categories

In [73]:
# Function to map emotion ids to broader categories and then to integers
def map_emotions(data):

    # Define a mapping from the 28 emotions to six broader categories
    emotion_mapping = {
        "admiration": "positive_intent", "amusement": "positive_intent", "anger": "negative_intent", "annoyance": "negative_intent",
        "approval": "positive_intent", "caring": "positive_intent", "confusion": "inquiry", "curiosity": "inquiry",
        "desire": "positive_intent", "disappointment": "negative_intent", "disapproval": "negative_intent", "disgust": "negative_intent",
        "embarrassment": "negative_intent", "excitement": "positive_intent", "fear": "urgency", "gratitude": "positive_intent",
        "grief": "negative_intent", "joy": "positive_intent", "love": "positive_intent", "nervousness": "urgency",
        "optimism": "positive_intent", "pride": "positive_intent", "realization": "inquiry", "relief": "positive_intent",
        "remorse": "negative_intent", "sadness": "negative_intent", "surprise": "positive_intent", "neutral": "neutral"
    }

    # Define a mapping from broader categories to integers
    category_to_int_mapping = {
        "neutral": 0, "negative_intent": 1, "positive_intent": 2, "inquiry": 3, "urgency": 4
    }

    # Emotions dictionary inside the function for mapping
    def map_emotion(emotion_id):
        original_emotion = emotions_dict.get(int(emotion_id), 'other')
        category = emotion_mapping.get(original_emotion, 'other')
        return category_to_int_mapping.get(category, -1)  # Use -1 for any unmapped categories

    data['emotion'] = data['emotion'].apply(map_emotion)
    return data

map_emotions_train_data = map_emotions(preprocessed_data)
print(map_emotions_train_data['emotion'].unique())
map_emotions_train_data.head(4)

[0 1 4 2 3]


,comment,emotion
0,"Now if he does off himself, everyone will thin...",0
1,WHY THE FUCK IS BAYLESS ISOING,1
2,To make her feel threatened,4
3,Dirty Southern Wankers,1
